In [1]:
# Import the findspark module to help locate and initialize Spark
import findspark

# Initialize Spark
findspark.init()

# Import the jupyter_black module, which is an extension to format code cells in Jupyter Notebook
import jupyter_black

# Load and enable the jupyter_black extension to format code cells automatically
jupyter_black.load()

In [2]:
# importing required libraries

import pandas as pd

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, count, isnan, when

from common import *

In [3]:
spark = SparkSession.builder.appName("NYCJobsDataKPIAnalysis").getOrCreate()

In [4]:
!ls -l "../../dataset/processed/"

total 0
drwxr-xr-x 1 root root 512 Aug 30 22:59 nyc_job_postings_processed_data.csv
drwxr-xr-x 1 root root 512 Aug 31 08:25 nyc_job_postings_processed_data.parquet


In [5]:
# load the processed dataset

df = spark.read.parquet(
    "../../dataset/processed/nyc_job_postings_processed_data.parquet"
)

In [7]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import desc, sum
from pyspark.sql.session import SparkSession


def get_top10_job_postings_per_category(df: DataFrame) -> DataFrame:
    """
    Get the top 10 job postings per category based on the number of positions.

    Args:
        df (DataFrame): The input DataFrame containing job posting data.

    Returns:
        DataFrame: A DataFrame with the top 10 job postings per category.
    """
    # Group by job category and aggregate the sum of positions
    category_positions = df.groupBy("Job_Category").agg(
        sum("#_Of_Positions").alias("Total_Positions")
    )

    # Sort by total positions in descending order and limit to top 10
    top10_categories = category_positions.sort(desc("Total_Positions")).limit(10)

    return top10_categories

In [8]:
top10_categories_df = get_top10_job_postings_per_category(df)
top10_categories_df.show(truncate=False)

+-----------------------------------------+---------------+
|Job_Category                             |Total_Positions|
+-----------------------------------------+---------------+
|Public Safety, Inspections, & Enforcement|1285           |
|Building Operations & Maintenance        |741            |
|Engineering, Architecture, & Planning    |665            |
|Technology, Data & Innovation            |400            |
|Health                                   |303            |
|Legal Affairs                            |288            |
|Finance, Accounting, & Procurement       |251            |
|Policy, Research & Analysis              |179            |
|Administration & Human Resources         |175            |
|Constituent Services & Community Programs|145            |
+-----------------------------------------+---------------+



In [41]:
#!python -m pip install plotly

In [44]:
tmp = top10_categories_df.toPandas()

In [51]:
import plotly.io as pio
import plotly.express as px

pio.templates.default = "presentation"

In [58]:
import plotly.express as px
import plotly.graph_objects as go


def create_annotated_bar_plot(data_frame, x, y, title):
    # Create a bar plot
    fig = px.bar(
        data_frame,
        x=x,
        y=y,
        title=title,
    )

    # Add annotations to the bars
    for index, row in data_frame.iterrows():
        fig.add_annotation(
            x=row[x],
            y=row[y],
            text=str(row[y]),
            showarrow=True,
            arrowhead=1,
        )

    # Customize layout
    fig.update_layout(
        xaxis_title=x,
        yaxis_title=y,
        xaxis_tickangle=-45,
        bargap=0.2,
    )

    fig.show()

In [59]:
# Call the function with the DataFrame
create_annotated_bar_plot(
    tmp, x="Job_Category", y="Total_Positions", title="Total Jobs per Category"
)

In [ ]:
# 2. Whats the salary distribution per job category?

In [10]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import round, avg, col, stddev, min, max

from pyspark.sql import functions as F

In [11]:
def calculate_salary_distribution(df: DataFrame) -> DataFrame:
    """
    Calculate salary distribution per job category.

    Args:
        df (DataFrame): Input DataFrame with the provided schema.

    Returns:
        DataFrame: DataFrame with rounded salary distribution statistics.
    """
    # Calculate salary distribution per job category
    salary_distribution = df.groupBy("Job_Category").agg(
        round(avg("AverageAnnualSalary"), 2).alias("AvgSalary"),
        round(stddev("AverageAnnualSalary"), 2).alias("SalaryStdDev"),
        round(min("AverageAnnualSalary"), 2).alias("MinSalary"),
        round(max("AverageAnnualSalary"), 2).alias("MaxSalary"),
    )

    return salary_distribution

In [12]:
# Calculate salary distribution
result_df = calculate_salary_distribution(df)

# Show the result
result_df.show()

+--------------------+---------+------------+---------+---------+
|        Job_Category|AvgSalary|SalaryStdDev|MinSalary|MaxSalary|
+--------------------+---------+------------+---------+---------+
|Administration & ...|  95000.0|         NaN|  95000.0|  95000.0|
|Health Policy, Re...| 128694.5|    39035.23|  94889.0| 162500.0|
|Administration & ...|  69040.5|         0.0|  69040.5|  69040.5|
|Information Techn...|  76941.5|         0.0|  76941.5|  76941.5|
|Finance, Accounti...|  63024.5|         NaN|  63024.5|  63024.5|
|Engineering, Arch...| 84286.67|    15379.16|  69283.5| 103050.0|
|Legal Affairs Pol...| 88898.67|     25122.4|  59705.5| 115732.0|
|Administration & ...|  53075.5|         0.0|  53075.5|  53075.5|
|Constituent Servi...| 67248.04|    17156.25|  31371.5| 135000.0|
|Building Operatio...| 71574.29|    30686.79|  36267.5| 151795.0|
|Engineering, Arch...| 128247.5|         0.0| 128247.5| 128247.5|
|Constituent Servi...| 61939.25|      4089.1|  56463.5|  67029.5|
|Administr

In [14]:
## 3. Is there any correlation between the higher degree and the salary?

In [15]:
# Group by "HighestDegree" and calculate the average salary

avg_salary_by_degree = df.groupBy("HighestDegree").agg(
    avg("AnnualSalaryTo").alias("AvgSalary")
)

# Show the result
avg_salary_by_degree.show()

+-------------+------------------+
|HighestDegree|         AvgSalary|
+-------------+------------------+
|         null| 95860.78691102262|
|       master|107468.30012121213|
|baccalaureate| 91239.54532382311|
|  high school| 85775.66294736842|
|      diploma| 58187.09613913044|
+-------------+------------------+



In [16]:
### 4: Whats the job posting having the highest salary per agency?

In [17]:
df = df.drop_duplicates()

In [18]:
# Remove duplicates based on the "Job ID" column
df = df.dropDuplicates(["Job_ID"])

In [19]:
from pyspark.sql.functions import col, max, count
from pyspark.sql.window import Window

# Define a window specification to partition by "Agency"
window_spec = Window.partitionBy("Agency")

# Calculate the maximum salary within each agency using the window function
df_with_max_salary = df.withColumn(
    "MaxSalary", max(col("AnnualSalaryTo")).over(window_spec)
)

# Filter rows where the salary matches the maximum salary per agency
result = df_with_max_salary.filter(col("AnnualSalaryTo") == col("MaxSalary"))

# Select the relevant columns for the final result
final_result = result.select("Job_ID", "Agency", "AnnualSalaryTo")

# Remove duplicates from the result
# final_result = final_result.distinct()

# Show the result
final_result.orderBy("AnnualSalaryTo").show(truncate=False)

+------+------------------------------+--------------+
|Job_ID|Agency                        |AnnualSalaryTo|
+------+------------------------------+--------------+
|170989|OFFICE OF COLLECTIVE BARGAININ|21548.8       |
|369120|MANHATTAN COMMUNITY BOARD #8  |39520.0       |
|399955|PUBLIC ADMINISTRATOR-NEW YORK |49349.0       |
|420065|EQUAL EMPLOY PRACTICES COMM   |72712.0       |
|420740|TEACHERS RETIREMENT SYSTEM    |75760.0       |
|404501|BOROUGH PRESIDENT-QUEENS      |77410.0       |
|425908|DEPARTMENT OF BUILDINGS       |82137.0       |
|417137|OFFICE OF EMERGENCY MANAGEMENT|85000.0       |
|405135|DEPT. OF HOMELESS SERVICES    |92001.0       |
|415959|NYC DEPT OF VETERANS' SERVICES|92001.0       |
|417207|ADMIN TRIALS AND HEARINGS     |100000.0      |
|403516|BUSINESS INTEGRITY COMMISSION |100000.0      |
|387034|BOARD OF CORRECTION           |102936.0      |
|402533|DEPT OF YOUTH & COMM DEV SRVS |110000.0      |
|416149|OFF OF PAYROLL ADMINISTRATION |115000.0      |
|423972|OF

In [20]:
from pyspark.sql.functions import col, max, count
from pyspark.sql.window import Window

# Find the job posting with the highest salary per agency
max_salary_per_agency = df.groupBy("Agency").agg(
    max("AverageAnnualSalary").alias("MaxSalary")
)
result = df.join(max_salary_per_agency, on="Agency").filter(
    col("AverageAnnualSalary") == col("MaxSalary")
)

# Remove duplicates from the result
final_result = result.distinct()

# Show the result
final_result.select("Job_ID", "Agency", "MaxSalary").orderBy("AnnualSalaryTo").show(
    truncate=False
)

+------+------------------------------+---------+
|Job_ID|Agency                        |MaxSalary|
+------+------------------------------+---------+
|170989|OFFICE OF COLLECTIVE BARGAININ|19874.4  |
|369120|MANHATTAN COMMUNITY BOARD #8  |39520.0  |
|399955|PUBLIC ADMINISTRATOR-NEW YORK |43296.0  |
|420065|EQUAL EMPLOY PRACTICES COMM   |67970.0  |
|420740|TEACHERS RETIREMENT SYSTEM    |69078.5  |
|404501|BOROUGH PRESIDENT-QUEENS      |65505.0  |
|425908|DEPARTMENT OF BUILDINGS       |76780.0  |
|417137|OFFICE OF EMERGENCY MANAGEMENT|70995.0  |
|415959|NYC DEPT OF VETERANS' SERVICES|82653.0  |
|405135|DEPT. OF HOMELESS SERVICES    |82653.0  |
|403516|BUSINESS INTEGRITY COMMISSION |95000.0  |
|417207|ADMIN TRIALS AND HEARINGS     |90000.0  |
|387034|BOARD OF CORRECTION           |96222.5  |
|424963|DEPT OF YOUTH & COMM DEV SRVS |91965.5  |
|423999|DEPARTMENT OF BUSINESS SERV.  |101097.0 |
|416149|OFF OF PAYROLL ADMINISTRATION |85995.0  |
|423972|OFFICE OF MANAGEMENT & BUDGET |117810.0 |


In [21]:
### 5. Whats the job positings average salary per agency for the last 2 years?

In [22]:
df.columns

['job_id',
 'agency',
 'posting_type',
 '#_of_positions',
 'business_title',
 'job_category',
 'Posting_Date',
 'annualsalaryfrom',
 'annualsalaryto',
 'averageannualsalary',
 'degrees',
 'highestdegree',
 'ngrams']

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, current_date, date_sub, avg

# Convert "Posting Date" to date format
df = df.withColumn("Posting_Date", col("Posting_Date").cast("date"))

nyears = 4

# Calculate the date 2 years ago from current date
two_years_ago = date_sub(current_date(), 365 * nyears)

# Filter data for the last 2 years
filtered_df = df.filter(col("Posting_Date") >= two_years_ago)

filtered_df.groupBy("Agency").agg(avg("AnnualSalaryTo")).show(2)

+--------------------+-------------------+
|              Agency|avg(AnnualSalaryTo)|
+--------------------+-------------------+
|     FIRE DEPARTMENT|  92683.63186666668|
|ADMIN FOR CHILDRE...|  81046.33333333333|
+--------------------+-------------------+
only showing top 2 rows



In [24]:
### 6. What are the highest paid skills in the US market?

In [26]:
df.columns

['job_id',
 'agency',
 'posting_type',
 '#_of_positions',
 'business_title',
 'job_category',
 'Posting_Date',
 'annualsalaryfrom',
 'annualsalaryto',
 'averageannualsalary',
 'degrees',
 'highestdegree',
 'ngrams']

In [30]:
from pyspark.sql.functions import col, split

# Split the 'name' column by comma and create a new column 'name_parts'
df = df.withColumn("ngrams_list", split(col("ngrams"), ","))

In [34]:
from pyspark.sql.functions import explode, round

exploded_df = df.select(explode("ngrams_list").alias("ngram"), "AnnualSalaryTo")

In [38]:
ngram_avg_salary_df = exploded_df.groupBy("ngram").agg(
    round(avg("AnnualSalaryTo"), 0).alias("avg_salary")
)

sorted_ngram_avg_salary_df = ngram_avg_salary_df.orderBy("avg_salary", ascending=False)

In [39]:
sorted_ngram_avg_salary_df.show(50, truncate=False)

+-----------------------+----------+
|ngram                  |avg_salary|
+-----------------------+----------+
|cciso                  |234402.0  |
|cgeit                  |234402.0  |
|multitasker            |225217.0  |
|extraordinary          |225217.0  |
|instituting            |225217.0  |
|solutionsdriven        |219746.0  |
|alignment              |218587.0  |
|mbaadvanced            |218587.0  |
|recruit                |218587.0  |
|resultsdriven          |218587.0  |
|concerning             |217244.0  |
|leasing                |217244.0  |
|multidivisional        |217244.0  |
|ipd                    |217244.0  |
|recertifications       |217244.0  |
|buildingleed           |217244.0  |
|counterpart            |214086.0  |
|sustaining             |214086.0  |
|clock                  |209585.0  |
|aligned                |209585.0  |
|sustain                |209585.0  |
|largest                |209585.0  |
|ethical                |202744.0  |
|cando                  |202744.0  |
|